# IBM Applied Data Science Capstone

## Finding a best location to organize an International Science Conference

### Import Libraries, Define Foursquare Credentials, the city and get its coordinates (latitude & longitude)
### Search for Hotels, Restaurants, Cafeteria & Shopping Malls and prepare the dataframes
### Generate map to visualize Hotels, Restaurants, Cafeterias and Shopping Malls and cluster together

In [2]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [3]:
ClIENT_ID = 'CBCRHQNLTE3PAHOGGHPKOQYRNHRRT1HV23G0SIRR1AOHKYEZ' # your Foursquare ID
ClIENT_SECRET = 'T0G1O0ODFLFSIRJGYK5NDHYYZZILPHZY1DHDSPZXKAXWANZ0' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: CBCRHQNLTE3PAHOGGHPKOQYRNHRRT1HV23G0SIRR1AOHKYEZ
Foursquare_Secret:T0G1O0ODFLFSIRJGYK5NDHYYZZILPHZY1DHDSPZXKAXWANZ0


In [4]:
# define the city and get its latitude & longitude 
city = 'Los Angeles'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

34.0536909 -118.2427666


## Segmenting Hotels

In [5]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CBCRHQNLTE3PAHOGGHPKOQYRNHRRT1HV23G0SIRR1AOHKYEZ&client_secret=T0G1O0ODFLFSIRJGYK5NDHYYZZILPHZY1DHDSPZXKAXWANZ0&ll=34.0536909,-118.2427666&v=20180604&query=Hotel&radius=500&limit=30'

In [6]:
# Send the GET Request and examine the results
results = requests.get(url).json()
#results

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4d954b06a243a5684965b473', 'name': 'R...",False,505db930e0e2e3d789c50aeb,The Palm Woods Hotel,US,Los Angeles,United States,Hollywood,86,"[The Palm Woods Hotel (Hollywood), Los Angeles...","[{'label': 'display', 'lat': 34.05403721919709...",34.054037,-118.241923,NaN,90012,CA,The Palm Woods Hotel,v-1590707845,NaN
1,"[{'id': '4c38df4de52ce0d596b336e1', 'name': 'P...",False,4cd50b6876ab721ebce76a6a,120 S Los Angeles St,US,Los Angeles,United States,NaN,352,"[120 S Los Angeles St, Los Angeles, CA 90012, ...","[{'label': 'display', 'lat': 34.050526, 'lng':...",34.050526,-118.242682,NaN,90012,CA,Kyoto Grand Hotel Parking & Gardens Garage,v-1590707845,NaN
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4a47ab0ff964a520eca91fe3,328 E 1st St,US,Los Angeles,United States,NaN,502,"[328 E 1st St, Los Angeles, CA 90012, United S...","[{'label': 'display', 'lat': 34.04974909864657...",34.049749,-118.240111,NaN,90012,CA,Miyako Hotel,v-1590707845,NaN
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,56292e5f498efdb6cec7a40a,NaN,US,Los Angeles,United States,NaN,107,"[Los Angeles, CA, United States]","[{'label': 'display', 'lat': 34.05304336547851...",34.053043,-118.241898,NaN,NaN,CA,Hotel Cortez,v-1590707845,NaN
4,[],False,4c95fb98efecb713c816e89d,5757 Telegraph Rd,US,Commerce,United States,NaN,74,"[5757 Telegraph Rd, Commerce, CA 90040, United...","[{'label': 'display', 'lat': 34.05333536922002...",34.053335,-118.243448,NaN,90040,CA,Wyndham Garden Hotel,v-1590707845,NaN


In [8]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Palm Woods Hotel,Residential Building (Apartment / Condo),The Palm Woods Hotel,US,Los Angeles,United States,Hollywood,86,"[The Palm Woods Hotel (Hollywood), Los Angeles...","[{'label': 'display', 'lat': 34.05403721919709...",34.054037,-118.241923,NaN,90012,CA,505db930e0e2e3d789c50aeb
1,Kyoto Grand Hotel Parking & Gardens Garage,Parking,120 S Los Angeles St,US,Los Angeles,United States,NaN,352,"[120 S Los Angeles St, Los Angeles, CA 90012, ...","[{'label': 'display', 'lat': 34.050526, 'lng':...",34.050526,-118.242682,NaN,90012,CA,4cd50b6876ab721ebce76a6a
2,Miyako Hotel,Hotel,328 E 1st St,US,Los Angeles,United States,NaN,502,"[328 E 1st St, Los Angeles, CA 90012, United S...","[{'label': 'display', 'lat': 34.04974909864657...",34.049749,-118.240111,NaN,90012,CA,4a47ab0ff964a520eca91fe3
3,Hotel Cortez,Hotel,NaN,US,Los Angeles,United States,NaN,107,"[Los Angeles, CA, United States]","[{'label': 'display', 'lat': 34.05304336547851...",34.053043,-118.241898,NaN,NaN,CA,56292e5f498efdb6cec7a40a
4,Wyndham Garden Hotel,None,5757 Telegraph Rd,US,Commerce,United States,NaN,74,"[5757 Telegraph Rd, Commerce, CA 90040, United...","[{'label': 'display', 'lat': 34.05333536922002...",34.053335,-118.243448,NaN,90040,CA,4c95fb98efecb713c816e89d


In [9]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,The Palm Woods Hotel,Residential Building (Apartment / Condo),The Palm Woods Hotel,34.054037,-118.241923,90012,CA
1,Kyoto Grand Hotel Parking & Gardens Garage,Parking,120 S Los Angeles St,34.050526,-118.242682,90012,CA
2,Miyako Hotel,Hotel,328 E 1st St,34.049749,-118.240111,90012,CA
3,Hotel Cortez,Hotel,NaN,34.053043,-118.241898,NaN,CA
4,Wyndham Garden Hotel,None,5757 Telegraph Rd,34.053335,-118.243448,90040,CA
5,Roes hotel,Hotel,2683 Euclid Avenue,34.052979,-118.243178,90001,CA
6,Hotel Cafe - 2nd Stage,Music Venue,NaN,34.052234,-118.243685,NaN,CA
7,Los Angeles City Hall,City Hall,200 N Spring St,34.053484,-118.242478,90012,CA
8,Kyoto Grand Hotel Garden,Garden,The Kyoto Grand Hotel and Gardens,34.051775,-118.242175,90012,CA
9,Little Toyko Hotel,Hotel,327 1/2 E 1st St,34.050409,-118.240011,90012,CA


In [10]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,The Palm Woods Hotel,Residential Building (Apartment / Condo),The Palm Woods Hotel,34.054037,-118.241923,90012,CA
1,Kyoto Grand Hotel Parking & Gardens Garage,Parking,120 S Los Angeles St,34.050526,-118.242682,90012,CA
2,Miyako Hotel,Hotel,328 E 1st St,34.049749,-118.240111,90012,CA
5,Roes hotel,Hotel,2683 Euclid Avenue,34.052979,-118.243178,90001,CA
7,Los Angeles City Hall,City Hall,200 N Spring St,34.053484,-118.242478,90012,CA
8,Kyoto Grand Hotel Garden,Garden,The Kyoto Grand Hotel and Gardens,34.051775,-118.242175,90012,CA
9,Little Toyko Hotel,Hotel,327 1/2 E 1st St,34.050409,-118.240011,90012,CA
10,Little Tokyo Hotel,Boarding House,327 1/2 E 1st St,34.050072,-118.239983,90012,CA
12,St George Hotel,Government Building,115 E 3rd St Apt 516,34.049488,-118.246138,90013,CA
13,Kawada Hotel,Hotel,200 S Hill St,34.052870,-118.247909,90012,CA


In [11]:
# delete rows which its category is not Hotel or Event Space
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
2,Miyako Hotel,Hotel,328 E 1st St,34.049749,-118.240111,90012,CA
5,Roes hotel,Hotel,2683 Euclid Avenue,34.052979,-118.243178,90001,CA
9,Little Toyko Hotel,Hotel,327 1/2 E 1st St,34.050409,-118.240011,90012,CA
13,Kawada Hotel,Hotel,200 S Hill St,34.052870,-118.247909,90012,CA
15,DoubleTree by Hilton,Hotel,120 South Los Angeles Street,34.050352,-118.242768,90012,CA


In [12]:
# delete rows which has duplicate hotel's name
df_hotels = hotel_dataframe.drop_duplicates(subset='name', keep="first")
df_hotels

,name,categories,address,lat,lng,postalCode,state
2,Miyako Hotel,Hotel,328 E 1st St,34.049749,-118.240111,90012,CA
5,Roes hotel,Hotel,2683 Euclid Avenue,34.052979,-118.243178,90001,CA
9,Little Toyko Hotel,Hotel,327 1/2 E 1st St,34.050409,-118.240011,90012,CA
13,Kawada Hotel,Hotel,200 S Hill St,34.052870,-118.247909,90012,CA
15,DoubleTree by Hilton,Hotel,120 South Los Angeles Street,34.050352,-118.242768,90012,CA


In [13]:
# choose the hotel which has the same postalCode with the event space
df_hotel = df_hotels[df_hotels.postalCode == '90012']
df_hotel

,name,categories,address,lat,lng,postalCode,state
2,Miyako Hotel,Hotel,328 E 1st St,34.049749,-118.240111,90012,CA
9,Little Toyko Hotel,Hotel,327 1/2 E 1st St,34.050409,-118.240011,90012,CA
13,Kawada Hotel,Hotel,200 S Hill St,34.052870,-118.247909,90012,CA
15,DoubleTree by Hilton,Hotel,120 South Los Angeles Street,34.050352,-118.242768,90012,CA


## Segmenting Restaurents

In [14]:
# search for Restaurants
search_query = 'Restaurant'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CBCRHQNLTE3PAHOGGHPKOQYRNHRRT1HV23G0SIRR1AOHKYEZ&client_secret=T0G1O0ODFLFSIRJGYK5NDHYYZZILPHZY1DHDSPZXKAXWANZ0&ll=34.0536909,-118.2427666&v=20180604&query=Restaurant&radius=10000&limit=30'

In [15]:
# Send the GET Request and examine the results
Rresults = requests.get(url).json()
#Rresults

In [16]:
# assign relevant part of JSON to venues
venues = Rresults['response']['venues']

# tranform venues into a dataframe
Restaurant_dataframe = json_normalize(venues)
Restaurant_dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",NaN,NaN,NaN,NaN,NaN,NaN,False,4aa8416bf964a5206c5020e3,800 N Alameda St,...,"[800 N Alameda St (in Union Station, along mai...","[{'label': 'display', 'lat': 34.05623229440184...",34.056232,-118.236183,NaN,90012,CA,Traxx Restaurant,v-1590707889,89373347
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,49ebc74af964a5202b671fe3,819 N Broadway,...,"[819 N Broadway (btwn College & Alpine), Los A...","[{'label': 'display', 'lat': 34.06292584487055...",34.062926,-118.238059,NaN,90012,CA,Yang Chow Restaurant,v-1590707889,48833648
2,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",1916080,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/mayflower-s...,False,4b046029f964a520b15322e3,679 N Spring St,...,"[679 N Spring St (Ord), Los Angeles, CA 90012,...","[{'label': 'display', 'lat': 34.05958203122075...",34.059582,-118.238129,NaN,90012,CA,Mayflower Seafood Restaurant,v-1590707889,NaN
3,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a7e6011f964a520d4f11fe3,251 S Olive St,...,"[251 S Olive St (at the Omni Hotel), Los Angel...","[{'label': 'display', 'lat': 34.05229026588889...",34.052290,-118.250963,NaN,90012,CA,Noe Restaurant & Bar,v-1590707889,502209411
4,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e31b01962e1d3ed6511373e,320 W 8th St,...,"[320 W 8th St, Los Angeles, CA 90014, United S...","[{'label': 'display', 'lat': 34.044552, 'lng':...",34.044552,-118.255757,NaN,90014,CA,Restaurant Guerrero,v-1590707889,NaN


In [17]:
# keep only columns that include venue name, and anything that is associated with location
Restaurant_clean_columns = ['name', 'categories'] + [col for col in Restaurant_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Restaurant_dataframe = Restaurant_dataframe.loc[:,Restaurant_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

# filter the category for each row
clean_Restaurant_dataframe['categories'] = clean_Restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_Restaurant_dataframe.columns]

clean_Restaurant_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Traxx Restaurant,New American Restaurant,800 N Alameda St,US,Los Angeles,United States,"in Union Station, along main corridor",669,"[800 N Alameda St (in Union Station, along mai...","[{'label': 'display', 'lat': 34.05623229440184...",34.056232,-118.236183,NaN,90012,CA,4aa8416bf964a5206c5020e3
1,Yang Chow Restaurant,Chinese Restaurant,819 N Broadway,US,Los Angeles,United States,btwn College & Alpine,1115,"[819 N Broadway (btwn College & Alpine), Los A...","[{'label': 'display', 'lat': 34.06292584487055...",34.062926,-118.238059,NaN,90012,CA,49ebc74af964a5202b671fe3
2,Mayflower Seafood Restaurant,Seafood Restaurant,679 N Spring St,US,Los Angeles,United States,Ord,782,"[679 N Spring St (Ord), Los Angeles, CA 90012,...","[{'label': 'display', 'lat': 34.05958203122075...",34.059582,-118.238129,NaN,90012,CA,4b046029f964a520b15322e3
3,Noe Restaurant & Bar,French Restaurant,251 S Olive St,US,Los Angeles,United States,at the Omni Hotel,771,"[251 S Olive St (at the Omni Hotel), Los Angel...","[{'label': 'display', 'lat': 34.05229026588889...",34.052290,-118.250963,NaN,90012,CA,4a7e6011f964a520d4f11fe3
4,Restaurant Guerrero,Mexican Restaurant,320 W 8th St,US,Los Angeles,United States,NaN,1571,"[320 W 8th St, Los Angeles, CA 90014, United S...","[{'label': 'display', 'lat': 34.044552, 'lng':...",34.044552,-118.255757,NaN,90014,CA,4e31b01962e1d3ed6511373e


In [19]:
# delete unnecessary columns
clean_Restaurant_dataframe2= clean_Restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
clean_Restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Traxx Restaurant,New American Restaurant,800 N Alameda St,34.056232,-118.236183,90012,CA
1,Yang Chow Restaurant,Chinese Restaurant,819 N Broadway,34.062926,-118.238059,90012,CA
2,Mayflower Seafood Restaurant,Seafood Restaurant,679 N Spring St,34.059582,-118.238129,90012,CA
3,Noe Restaurant & Bar,French Restaurant,251 S Olive St,34.052290,-118.250963,90012,CA
4,Restaurant Guerrero,Mexican Restaurant,320 W 8th St,34.044552,-118.255757,90014,CA
5,"24/7 Restaurant at The Standard, Downtown LA",American Restaurant,550 S Flower St,34.049743,-118.256548,90071,CA
6,Trimana Restaurant,Salad Place,633 W 5th St.,34.050811,-118.254983,90071,CA
7,Full House Seafood Restaurant,Chinese Restaurant,963 N Hill St,34.066155,-118.237916,90012,CA
8,Restaurant Atitlan,Latin American Restaurant,1614 W 6th St,34.057095,-118.269879,90017,CA
9,Wok Inn Restaurant,Asian Restaurant,201 N Los Angeles St Ste 102,34.054317,-118.241752,90012,CA


In [20]:
# delete rows with none values
df_Restaurant = clean_Restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
0,Traxx Restaurant,New American Restaurant,800 N Alameda St,34.056232,-118.236183,90012,CA
1,Yang Chow Restaurant,Chinese Restaurant,819 N Broadway,34.062926,-118.238059,90012,CA
2,Mayflower Seafood Restaurant,Seafood Restaurant,679 N Spring St,34.059582,-118.238129,90012,CA
3,Noe Restaurant & Bar,French Restaurant,251 S Olive St,34.052290,-118.250963,90012,CA
4,Restaurant Guerrero,Mexican Restaurant,320 W 8th St,34.044552,-118.255757,90014,CA
5,"24/7 Restaurant at The Standard, Downtown LA",American Restaurant,550 S Flower St,34.049743,-118.256548,90071,CA
6,Trimana Restaurant,Salad Place,633 W 5th St.,34.050811,-118.254983,90071,CA
7,Full House Seafood Restaurant,Chinese Restaurant,963 N Hill St,34.066155,-118.237916,90012,CA
8,Restaurant Atitlan,Latin American Restaurant,1614 W 6th St,34.057095,-118.269879,90017,CA
9,Wok Inn Restaurant,Asian Restaurant,201 N Los Angeles St Ste 102,34.054317,-118.241752,90012,CA


## Segmenting Cafeterias

In [21]:
# search for Cafeteria
search_query = 'Cafeteria'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CBCRHQNLTE3PAHOGGHPKOQYRNHRRT1HV23G0SIRR1AOHKYEZ&client_secret=T0G1O0ODFLFSIRJGYK5NDHYYZZILPHZY1DHDSPZXKAXWANZ0&ll=34.0536909,-118.2427666&v=20180604&query=Cafeteria&radius=10000&limit=30'

In [22]:
# Send the GET Request and examine the results
cresults = requests.get(url).json()
#cresults

In [23]:
# assign relevant part of JSON to venues
venues = cresults['response']['venues']

# tranform venues into a dataframe
Cafeteria_dataframe = json_normalize(venues)
Cafeteria_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '54f4ba06498e2cf5561da814', 'name': 'C...",False,5c50b208e706550025100d65,350 W 1st St,US,Los Angeles,United States,Hill St,344,"[350 W 1st St (Hill St), Los Angeles, CA 90012...","[{'label': 'display', 'lat': 34.053543, 'lng':...",34.053543,-118.246495,Downtown Los Angeles,90012,CA,Central Dist Of Ca Cafeteria - Cube Cafe,v-1590707927
1,"[{'id': '4bf58dd8d48988d1a1941735', 'name': 'C...",False,4d42001faed776ebf42b6a97,1500 San Pablo St,US,Los Angeles,United States,NaN,3930,"[1500 San Pablo St, Los Angeles, CA 90033, Uni...","[{'label': 'display', 'lat': 34.06192862413815...",34.061929,-118.201323,NaN,90033,CA,Keck Hospital of USC Cafeteria (UNH),v-1590707927
2,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,50bfad93e4b0b6fd9fc6e55f,210 W Temple St,US,Los Angeles,United States,NaN,167,"[210 W Temple St, Los Angeles, CA 90012, Unite...","[{'label': 'display', 'lat': 34.05491252028065...",34.054913,-118.243825,NaN,90012,CA,Clara Shortridge Foltz Criminal Building Cafet...,v-1590707927
3,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,5298d210498ea4824841b8fa,300 N Los Angeles Street,US,Los Angeles,United States,NaN,247,"[300 N Los Angeles Street, Los Angeles, CA 900...","[{'label': 'display', 'lat': 34.05389609865689...",34.053896,-118.240091,NaN,90012,CA,Federal Building Cafeteria,v-1590707927
4,"[{'id': '4bf58dd8d48988d128941735', 'name': 'C...",False,4c460ad737f6d13a1dea6196,145 S Spring St,US,Los Angeles,United States,at 2nd St.,286,"[145 S Spring St (at 2nd St.), Los Angeles, CA...","[{'label': 'display', 'lat': 34.05200466638617...",34.052005,-118.245113,NaN,90012,CA,Los Angeles Times Cafeteria,v-1590707927


In [24]:
# keep only columns that include venue name, and anything that is associated with location
Cafeteria_clean_columns = ['name', 'categories'] + [col for col in Cafeteria_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Cafeteria_dataframe = Cafeteria_dataframe.loc[:,Cafeteria_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list4 = row['categories']
    except:
        categories_list4 = row['venue.categories']
        
    if len(categories_list4) == 0:
        return None
    else:
        return categories_list4[0]['name']

# filter the category for each row
clean_Cafeteria_dataframe['categories'] = clean_Cafeteria_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Cafeteria_dataframe.columns = [column.split('.')[-1] for column in clean_Cafeteria_dataframe.columns]

clean_Cafeteria_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Central Dist Of Ca Cafeteria - Cube Cafe,Corporate Cafeteria,350 W 1st St,US,Los Angeles,United States,Hill St,344,"[350 W 1st St (Hill St), Los Angeles, CA 90012...","[{'label': 'display', 'lat': 34.053543, 'lng':...",34.053543,-118.246495,Downtown Los Angeles,90012,CA,5c50b208e706550025100d65
1,Keck Hospital of USC Cafeteria (UNH),College Cafeteria,1500 San Pablo St,US,Los Angeles,United States,NaN,3930,"[1500 San Pablo St, Los Angeles, CA 90033, Uni...","[{'label': 'display', 'lat': 34.06192862413815...",34.061929,-118.201323,NaN,90033,CA,4d42001faed776ebf42b6a97
2,Clara Shortridge Foltz Criminal Building Cafet...,Cafeteria,210 W Temple St,US,Los Angeles,United States,NaN,167,"[210 W Temple St, Los Angeles, CA 90012, Unite...","[{'label': 'display', 'lat': 34.05491252028065...",34.054913,-118.243825,NaN,90012,CA,50bfad93e4b0b6fd9fc6e55f
3,Federal Building Cafeteria,Cafeteria,300 N Los Angeles Street,US,Los Angeles,United States,NaN,247,"[300 N Los Angeles Street, Los Angeles, CA 900...","[{'label': 'display', 'lat': 34.05389609865689...",34.053896,-118.240091,NaN,90012,CA,5298d210498ea4824841b8fa
4,Los Angeles Times Cafeteria,Cafeteria,145 S Spring St,US,Los Angeles,United States,at 2nd St.,286,"[145 S Spring St (at 2nd St.), Los Angeles, CA...","[{'label': 'display', 'lat': 34.05200466638617...",34.052005,-118.245113,NaN,90012,CA,4c460ad737f6d13a1dea6196


In [25]:
# delete unnecessary columns
clean_Cafeteria_dataframe2= clean_Cafeteria_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Cafeteria_dataframe2

,name,categories,address,lat,lng,neighborhood,postalCode,state
0,Central Dist Of Ca Cafeteria - Cube Cafe,Corporate Cafeteria,350 W 1st St,34.053543,-118.246495,Downtown Los Angeles,90012,CA
1,Keck Hospital of USC Cafeteria (UNH),College Cafeteria,1500 San Pablo St,34.061929,-118.201323,NaN,90033,CA
2,Clara Shortridge Foltz Criminal Building Cafet...,Cafeteria,210 W Temple St,34.054913,-118.243825,NaN,90012,CA
3,Federal Building Cafeteria,Cafeteria,300 N Los Angeles Street,34.053896,-118.240091,NaN,90012,CA
4,Los Angeles Times Cafeteria,Cafeteria,145 S Spring St,34.052005,-118.245113,NaN,90012,CA
5,Roybal Federal Building Cafeteria,Cafeteria,255 E Temple St,34.053107,-118.239222,NaN,90012,CA
6,School Cafeteria,Restaurant,2226 E 3rd St,34.042303,-118.213334,NaN,90033,CA
7,DWP Cafeteria,American Restaurant,100 Dewap St,34.057133,-118.249163,NaN,NaN,CA
8,Cafeteria Alma Rosa,Coffee Shop,727 S Spring St,34.044142,-118.252592,NaN,90014,CA
9,Metro HW Cafeteria,Cafeteria,NaN,34.056164,-118.234169,NaN,90012,CA


In [26]:
# delete rows with none values
df_Cafeteria = clean_Cafeteria_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Cafeteria

,name,categories,address,lat,lng,neighborhood,postalCode,state
0,Central Dist Of Ca Cafeteria - Cube Cafe,Corporate Cafeteria,350 W 1st St,34.053543,-118.246495,Downtown Los Angeles,90012,CA


## Segmenting Shopping Malls

In [27]:
# search for Shopping Malls
search_query = 'Shopping'
radius = 1000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=CBCRHQNLTE3PAHOGGHPKOQYRNHRRT1HV23G0SIRR1AOHKYEZ&client_secret=T0G1O0ODFLFSIRJGYK5NDHYYZZILPHZY1DHDSPZXKAXWANZ0&ll=34.0536909,-118.2427666&v=20180604&query=Shopping&radius=1000&limit=30'

In [28]:
# Send the GET Request and examine the results
sresults = requests.get(url).json()
#sresults

In [29]:
# assign relevant part of JSON to venues
venues = sresults['response']['venues']

# tranform venues into a dataframe
Shopping_dataframe = json_normalize(venues)
Shopping_dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c255030a852c928b1b7e46c,333 S Alameda St,...,3rd St,1016,"[333 S Alameda St (3rd St), Los Angeles, CA 90...","[{'label': 'display', 'lat': 34.04525390906229...",34.045254,-118.238537,90013,CA,Little Tokyo Galleria Shopping Center,v-1590707956
1,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,50a55204e4b0551f8a4ced2f,NaN,...,NaN,1256,"[Los Angeles, CA 90012, United States]","[{'label': 'display', 'lat': 34.064115, 'lng':...",34.064115,-118.237529,90012,CA,Grape Shopping In China Town,v-1590707956
2,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",1128330,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/little-toky...,False,4a2dcdf8f964a520f6971fe3,333 S Alameda St #100,...,at E 3rd St,1066,"[333 S Alameda St #100 (at E 3rd St), Los Ange...","[{'label': 'display', 'lat': 34.04474630692715...",34.044746,-118.238639,90013,CA,Little Tokyo Market Place,v-1590707956
3,"[{'id': '4f04afc02fb6e1c99f3db0bc', 'name': 'M...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c7e8e318ae376b0f8868774,333 S Alameda St,...,3rd Street,1014,"[333 S Alameda St (3rd Street), Los Angeles, C...","[{'label': 'display', 'lat': 34.04526508172578...",34.045265,-118.238579,90013,CA,Sprint PCS Kiosk,v-1590707956
4,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",2082353,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/the-pho-sho...,False,4c0c279a6071a593a463e232,333 S Alameda St Ste 125,...,at Little Tokyo Galleria Shopping Center,1028,[333 S Alameda St Ste 125 (at Little Tokyo Gal...,"[{'label': 'display', 'lat': 34.04513951801520...",34.045140,-118.238557,90013,CA,The Pho Shop,v-1590707956


In [30]:
# keep only columns that include venue name, and anything that is associated with location
Shopping_clean_columns = ['name', 'categories'] + [col for col in Shopping_dataframe.columns if col.startswith('location.')]+ ['id']
clean_Shopping_dataframe = Shopping_dataframe.loc[:,Shopping_clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list5 = row['categories']
    except:
        categories_list5 = row['venue.categories']
        
    if len(categories_list5) == 0:
        return None
    else:
        return categories_list5[0]['name']

# filter the category for each row
clean_Shopping_dataframe['categories'] = clean_Shopping_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_Shopping_dataframe.columns = [column.split('.')[-1] for column in clean_Shopping_dataframe.columns]

clean_Shopping_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Little Tokyo Galleria Shopping Center,Shopping Mall,333 S Alameda St,US,Los Angeles,United States,3rd St,1016,"[333 S Alameda St (3rd St), Los Angeles, CA 90...","[{'label': 'display', 'lat': 34.04525390906229...",34.045254,-118.238537,90013,CA,4c255030a852c928b1b7e46c
1,Grape Shopping In China Town,Food Truck,NaN,US,Los Angeles,United States,NaN,1256,"[Los Angeles, CA 90012, United States]","[{'label': 'display', 'lat': 34.064115, 'lng':...",34.064115,-118.237529,90012,CA,50a55204e4b0551f8a4ced2f
2,Little Tokyo Market Place,Grocery Store,333 S Alameda St #100,US,Los Angeles,United States,at E 3rd St,1066,"[333 S Alameda St #100 (at E 3rd St), Los Ange...","[{'label': 'display', 'lat': 34.04474630692715...",34.044746,-118.238639,90013,CA,4a2dcdf8f964a520f6971fe3
3,Sprint PCS Kiosk,Mobile Phone Shop,333 S Alameda St,US,Los Angeles,United States,3rd Street,1014,"[333 S Alameda St (3rd Street), Los Angeles, C...","[{'label': 'display', 'lat': 34.04526508172578...",34.045265,-118.238579,90013,CA,4c7e8e318ae376b0f8868774
4,The Pho Shop,Asian Restaurant,333 S Alameda St Ste 125,US,Los Angeles,United States,at Little Tokyo Galleria Shopping Center,1028,[333 S Alameda St Ste 125 (at Little Tokyo Gal...,"[{'label': 'display', 'lat': 34.04513951801520...",34.045140,-118.238557,90013,CA,4c0c279a6071a593a463e232


In [31]:
# delete unnecessary columns
clean_Shopping_dataframe2= clean_Shopping_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
clean_Shopping_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Little Tokyo Galleria Shopping Center,Shopping Mall,333 S Alameda St,34.045254,-118.238537,90013,CA
1,Grape Shopping In China Town,Food Truck,NaN,34.064115,-118.237529,90012,CA
2,Little Tokyo Market Place,Grocery Store,333 S Alameda St #100,34.044746,-118.238639,90013,CA
3,Sprint PCS Kiosk,Mobile Phone Shop,333 S Alameda St,34.045265,-118.238579,90013,CA
4,The Pho Shop,Asian Restaurant,333 S Alameda St Ste 125,34.045140,-118.238557,90013,CA


In [32]:
# delete rows which its category is not Shopping Mall
df_Shopping = clean_Shopping_dataframe2[clean_Shopping_dataframe2.categories == 'Shopping Mall']
df_Shopping

,name,categories,address,lat,lng,postalCode,state
0,Little Tokyo Galleria Shopping Center,Shopping Mall,333 S Alameda St,34.045254,-118.238537,90013,CA


## Clustering Hotels, Restaurants, Cafeterias and Shopping Malls

In [33]:
# create dataframe of Hotels, Restaurants, Cafeterias and Shopping Malls
hotel_neighbourhood_df = pd.concat([df_hotel, df_Restaurant, df_Cafeteria, df_Shopping], ignore_index=True)
hotel_neighbourhood_df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


,address,categories,lat,lng,name,neighborhood,postalCode,state
0,328 E 1st St,Hotel,34.049749,-118.240111,Miyako Hotel,NaN,90012,CA
1,327 1/2 E 1st St,Hotel,34.050409,-118.240011,Little Toyko Hotel,NaN,90012,CA
2,200 S Hill St,Hotel,34.052870,-118.247909,Kawada Hotel,NaN,90012,CA
3,120 South Los Angeles Street,Hotel,34.050352,-118.242768,DoubleTree by Hilton,NaN,90012,CA
4,800 N Alameda St,New American Restaurant,34.056232,-118.236183,Traxx Restaurant,NaN,90012,CA
5,819 N Broadway,Chinese Restaurant,34.062926,-118.238059,Yang Chow Restaurant,NaN,90012,CA
6,679 N Spring St,Seafood Restaurant,34.059582,-118.238129,Mayflower Seafood Restaurant,NaN,90012,CA
7,251 S Olive St,French Restaurant,34.052290,-118.250963,Noe Restaurant & Bar,NaN,90012,CA
8,320 W 8th St,Mexican Restaurant,34.044552,-118.255757,Restaurant Guerrero,NaN,90014,CA
9,550 S Flower St,American Restaurant,34.049743,-118.256548,"24/7 Restaurant at The Standard, Downtown LA",NaN,90071,CA


In [34]:
# Generate map to visualize Hotel neighbourhood including Restaurants, Cafeterias and Shopping Malls 
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighbourhood_df['lat'], hotel_neighbourhood_df['lng'], 
                                           hotel_neighbourhood_df['name'], hotel_neighbourhood_df['categories'],\
                                               hotel_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map